In [1]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import imutils
import sys
import numpy
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pickle
numpy.set_printoptions(threshold=sys.maxsize)

4.1.1


# Extract marks from images to dictionary and save

In [19]:
path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/"

def get_marks(filepath):
    marks = []
    img = cv2.imread(filepath)
    validation_sum = 3*255 - 50
    for x,y in np.ndindex(img.shape[:2]):
        if np.sum(img[(x,y)]) > validation_sum:
            marks.append((y,x))
    print(marks)
    return marks

def get_all_marks():
    marks = dict()
    filenames = [f for f in sorted(listdir(path)) if isfile(join(path, f))]
    for i in tqdm(range(len(filenames))):
        filename = filenames[i]
        print(path, filename)
        marks[filename] = get_marks(path + filename)
    return marks

marks = get_all_marks()
np.save(path+"eval/test_marks", marks)


  0%|          | 0/85 [00:00<?, ?it/s]

/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_0.jpg


  1%|          | 1/85 [00:01<02:08,  1.53s/it]

[(495, 264), (310, 266)]
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_1.jpg


  2%|▏         | 2/85 [00:03<02:07,  1.54s/it]

...

[(424, 311)]
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_2_8.jpg


 99%|█████████▉| 84/85 [02:02<00:01,  1.44s/it]

[(418, 315)]
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_2_9.jpg


100%|██████████| 85/85 [02:03<00:00,  1.45s/it]

[(397, 318)]


# Evaluate

In [11]:
def dist(a,b):
    return np.linalg.norm(np.array(a)-np.array(b))
    
path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/eval/"
pred = np.load(path+"pred_centers.npy", allow_pickle=True).item()
test = np.load(path+"test_marks.npy", allow_pickle=True).item()
filenames = test.keys()
deviations = []
missed_predictions = 0
well_predictions = 0
redundant_predictions = 0
acceptance_thr = 15
for filename in filenames:
    marks = test[filename]
    center = pred[filename][0]
    distances = [dist(center, x) for x in marks]
    if distances:
        print(distances)
        min_distance = int(np.min(distances))
        deviations.append(min_distance)
        if min_distance <= acceptance_thr:
            well_predictions += 1
        else:
            missed_predictions += 1
    else:
        redundant_predictions += 1
        
    
error = np.mean(deviations)
print(deviations, error, " well:",well_predictions, " missed:", missed_predictions, " redundant:", redundant_predictions)

[8.06225774829855, 186.09674903124989]
[6.576473218982953, 189.08000951977974]
[9.848857801796104, 190.09471323527123]
[187.08086486864443, 3.5]
[6.264982043070834, 196.03124750916626]
[7.905694150420948, 204.52994890724437]
[8.54400374531753, 228.0548179714693]
[1.118033988749895]
[2.5]
[1.0]
[7.106335201775948]
[6.946221994724902]
[7.615773105863909, 189.02380802428036]
[5.5901699437494745]
[145.98630072715727]
[6.5, 185.08173869941896]
[4.6097722286464435]
[1.5811388300841898]
[1.118033988749895]
[238.0021008310641, 1.4142135623730951]
[6.519202405202649, 207.50542161591827]
[203.0098519776811, 6.4031242374328485]
[404.5522215981517, 198.50566742539115, 6.041522986797286]
[163.8909698549618, 55.823382197785186, 250.0144995795244]
[212.31168126130035, 2.692582403567252, 207.04890726589213]
[226.2675407565124, 2.0]
[7.158910531638177, 190.07958859383086]
[127.54803016903084, 281.12719541161437]
[2.0615528128088303]
[4.301162633521313]
[2.692582403567252]
[4.716990566028302]
[153.70832